# [Classification task](https://www.kaggle.com/competitions/star-type-classification)

- **Vmag** - Визуальная видимая величина звезды
- **Plx** - Расстояние между звездой и Землей (параллакс)
- **e_Plx** - Стандартная ошибка параллакса (подсказка: если значение очень велико - это плохо, скорее всего такие объекты нужно отбросить)
- **B-V** - Индекс цвета. (Горячая звезда имеет показатель цвета B-V, близкий к 0 или отрицательный, тогда как холодная звезда имеет показатель цвета B-V, близкий к 2).
- **SpType** - Спектральный тип звезды по классификации МК
- **Amag** - Абсолютная величина звезды (Absolute Magnitude of the Star)
- **TargetClass** - Целевая переменная (Является ли звезда карликом (0) или гигантом (1))

## My code

In [24]:
import pandas as pd

In [25]:
train = pd.read_csv("train_star.csv")
test = pd.read_csv("test_star.csv")

In [26]:
import re
from sklearn.preprocessing import StandardScaler
import numpy as np

# Функции для обработки SpType
def parse_spectral_class(sptype):
    match = re.match(r'^([OBAFGKM])([^/]*)', sptype)
    return match.group(1) if match else 'Unknown'

def parse_subclass(sptype):
    match = re.search(r'(\d+\.?\d*)', sptype)
    return float(match.group(1)) if match else 0.0

def parse_luminosity_class(sptype):
    match = re.search(r'(II{1,2}|I{1,4}|IV|V)\b', sptype)
    return match.group(1) if match else 'V' if 'V' in sptype else 'Unknown'

# Предобработка данных
class DataPreprocessor:
    def __init__(self):
        self.scaler = None
        self.spectral_classes = ['O','B','A','F','G','K','M','Unknown']
        self.luminosity_classes = ['V','IV','III','II','I','Unknown']
    
    def fit_transform(self, df, is_train=True):
        df = df.copy()
        # Извлечение признаков
        df['spectral_class'] = df['SpType'].apply(parse_spectral_class)
        df['subclass'] = df['SpType'].apply(parse_subclass)
        df['luminosity_class'] = df['SpType'].apply(parse_luminosity_class)
        
        # Кодирование категориальных признаков
        df['spectral_class'] = pd.Categorical(df['spectral_class'], categories=self.spectral_classes)
        spectral_dummies = pd.get_dummies(df['spectral_class'], prefix='spec')
        
        df['luminosity_class'] = pd.Categorical(df['luminosity_class'], categories=self.luminosity_classes)
        luminosity_dummies = pd.get_dummies(df['luminosity_class'], prefix='lum')
        
        # Числовые признаки
        df['Plx_over_ePlx'] = df['Plx'] / df['e_Plx'].replace(0, np.nan)
        df['Plx_over_ePlx'] = df['Plx_over_ePlx'].fillna(0).replace([np.inf, -np.inf], 0)
        
        numerical = ['Vmag','Plx','e_Plx','B-V','Amag','subclass','Plx_over_ePlx']
        
        # Масштабирование
        if is_train:
            self.scaler = StandardScaler()
            df[numerical] = self.scaler.fit_transform(df[numerical])
        else:
            df[numerical] = self.scaler.transform(df[numerical])
            
        return pd.concat([df[numerical], spectral_dummies, luminosity_dummies], axis=1)

In [27]:
preprocessor = DataPreprocessor()

X_train = preprocessor.fit_transform(train.drop(columns=['TargetClass']), is_train=True)
y_train = (train["TargetClass"] == "Giant").astype(int)
X_test = preprocessor.fit_transform(test, is_train=False)

In [28]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_test = log_reg.predict(X_test)

In [29]:
pd.DataFrame(y_test).rename(columns={0: "TargetClass"}).reset_index().to_csv("prediction.csv", index=False)

## Deepseek code

У меня есть датасет без пропусков звезд с следующими параметрами:
- **Vmag** - Визуальная видимая величина звезды
- **Plx** - Расстояние между звездой и Землей (параллакс)
- **e_Plx** - Стандартная ошибка параллакса (подсказка: если значение очень велико - это плохо, скорее всего такие объекты нужно отбросить)
- **B-V** - Индекс цвета. (Горячая звезда имеет показатель цвета B-V, близкий к 0 или отрицательный, тогда как холодная звезда имеет показатель цвета B-V, близкий к 2).
- **SpType** - Спектральный тип звезды по классификации МК
- **Amag** - Абсолютная величина звезды (Absolute Magnitude of the Star)
- **TargetClass** - Целевая переменная (Является ли звезда карликом (0) или гигантом (1))

Мне нужно предсказать при помощи классификации TargetClass.
Команда `train["TargetClass"].value_counts()` выдает:
```
TargetClass
Giant    15793
Dwarf     4939
```
А `train["SpType"].unique()`:
```
array(['G5/G6V', 'M1V:', 'F3V', ..., 'B6Vwp...', 'G6:V:+...', 'K1V(p)'],
      dtype=object)
```
Длинны 1600.
Я ожидаю от тебя нейронную сеть наиболее подходящую для этой задачи, которая на другой часть датасета выдаст ответ.

In [30]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential, layers, callbacks, optimizers

# Функции для обработки SpType
def parse_spectral_class(sptype):
    match = re.match(r'^([OBAFGKM])([^/]*)', sptype)
    return match.group(1) if match else 'Unknown'

def parse_subclass(sptype):
    match = re.search(r'(\d+\.?\d*)', sptype)
    return float(match.group(1)) if match else 0.0

def parse_luminosity_class(sptype):
    match = re.search(r'(II{1,2}|I{1,4}|IV|V)\b', sptype)
    return match.group(1) if match else 'V' if 'V' in sptype else 'Unknown'

# Предобработка данных
class DataPreprocessor:
    def __init__(self):
        self.scaler = None
        self.spectral_classes = ['O','B','A','F','G','K','M','Unknown']
        self.luminosity_classes = ['V','IV','III','II','I','Unknown']
    
    def fit_transform(self, df, is_train=True):
        df = df.copy()
        # Извлечение признаков
        df['spectral_class'] = df['SpType'].apply(parse_spectral_class)
        df['subclass'] = df['SpType'].apply(parse_subclass)
        df['luminosity_class'] = df['SpType'].apply(parse_luminosity_class)
        
        # Кодирование категориальных признаков
        df['spectral_class'] = pd.Categorical(df['spectral_class'], categories=self.spectral_classes)
        spectral_dummies = pd.get_dummies(df['spectral_class'], prefix='spec')
        
        df['luminosity_class'] = pd.Categorical(df['luminosity_class'], categories=self.luminosity_classes)
        luminosity_dummies = pd.get_dummies(df['luminosity_class'], prefix='lum')
        
        # Числовые признаки
        df['Plx_over_ePlx'] = df['Plx'] / df['e_Plx'].replace(0, np.nan)
        df['Plx_over_ePlx'] = df['Plx_over_ePlx'].fillna(0).replace([np.inf, -np.inf], 0)
        
        numerical = ['Vmag','Plx','e_Plx','B-V','Amag','subclass','Plx_over_ePlx']
        
        # Масштабирование
        if is_train:
            self.scaler = StandardScaler()
            df[numerical] = self.scaler.fit_transform(df[numerical])
        else:
            df[numerical] = self.scaler.transform(df[numerical])
            
        return pd.concat([df[numerical], spectral_dummies, luminosity_dummies], axis=1)

In [31]:
# Загрузка данных
train = pd.read_csv("train_star.csv")
test = pd.read_csv("test_star.csv")

# Предобработка
preprocessor = DataPreprocessor()
X_train = preprocessor.fit_transform(train.drop(columns=['TargetClass']), is_train=True)
y_train = (train["TargetClass"] == "Giant").astype(int)
X_test = preprocessor.fit_transform(test, is_train=False)

In [32]:
# Построение модели
model = Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=optimizers.Adam(0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

# Балансировка классов
class_weight = {0: 2.5, 1: 0.7}
y_train.info()

# Обучение
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=256,
    validation_split=0.2,
    class_weight=class_weight,
    callbacks=[callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

<class 'pandas.core.series.Series'>
RangeIndex: 20732 entries, 0 to 20731
Series name: TargetClass
Non-Null Count  Dtype
--------------  -----
20732 non-null  int64
dtypes: int64(1)
memory usage: 162.1 KB
Epoch 1/100


/home/andredon/projects/ADA/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8136 - auc_2: 0.8657 - loss: 0.5363 - val_accuracy: 0.9395 - val_auc_2: 0.9835 - val_loss: 0.1699
Epoch 2/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9427 - auc_2: 0.9820 - loss: 0.1738 - val_accuracy: 0.9590 - val_auc_2: 0.9889 - val_loss: 0.1329
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - accuracy: 0.9597 - auc_2: 0.9872 - loss: 0.1152 - val_accuracy: 0.9605 - val_auc_2: 0.9901 - val_loss: 0.1211
Epoch 4/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.9628 - auc_2: 0.9877 - loss: 0.1100 - val_accuracy: 0.9605 - val_auc_2: 0.9908 - val_loss: 0.1184
Epoch 5/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step - accuracy: 0.9633 - auc_2: 0.9889 - loss: 0.0995 - val_accuracy: 0.9605 - val_auc_2: 0.9913 - val_loss: 0.1120
Epoch 6/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9658 - auc_2: 0.9891 - loss: 0.0958 - val_accuracy: 0.9605 - val_auc_2: 0.9912 - val_loss: 0.1141
Epoch 7/100
65/65 ━━━━━━━━━━

In [33]:
# Предсказание и сохранение
predictions = model.predict(X_test)
result = pd.DataFrame({
    'TargetClass': (predictions.squeeze() >= 0.5).astype(int)
})
pd.DataFrame(result).rename(columns={0: "TargetClass"}).reset_index().to_csv("predictions.csv", index=False)

589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 163us/step
